In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # TO-DO: Complete here
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    print_solution(data, manager, routing, solution)

In [5]:
N_items = [72,29,15,22]
weight_items = [20.5,20.4,87.48,16.5]
min_place = [90,90,90,90,20,20]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]

demand = [0]
for i in range(len(min_place)):
    demand.append(total_demand*min_place[i]/sum(min_place))
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Monday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

Route for vehicle 0:
 0 Load(0) ->  5 Load(187) ->  4 Load(1029) ->  3 Load(1871) ->  0 Load(1871)
Distance of the route: 11300m
Load of the route: 1871

Route for vehicle 1:
 0 Load(0) ->  1 Load(842) ->  2 Load(1684) ->  6 Load(1871) ->  0 Load(1871)
Distance of the route: 4950m
Load of the route: 1871

Total distance of all routes: 16250m
Total load of all routes: 3742


In [16]:
N_items = [58,26,9,11]
weight_items = [20.5,20.4,87.48,16.5]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]
print(total_demand)
demand = [0]
for i in range(24):
    demand.append(total_demand/24)
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Wednesday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

2688.2200000000003
Route for vehicle 0:
 0 Load(0) ->  17 Load(112) ->  3 Load(224) ->  15 Load(336) ->  16 Load(448) ->  24 Load(560) ->  18 Load(672) ->  4 Load(784) ->  5 Load(896) ->  22 Load(1008) ->  7 Load(1120) ->  6 Load(1232) ->  2 Load(1344) ->  23 Load(1456) ->  21 Load(1568) ->  0 Load(1568)
Distance of the route: 28150m
Load of the route: 1568

Route for vehicle 1:
 0 Load(0) ->  1 Load(112) ->  8 Load(224) ->  11 Load(336) ->  19 Load(448) ->  12 Load(560) ->  13 Load(672) ->  14 Load(784) ->  20 Load(896) ->  10 Load(1008) ->  9 Load(1120) ->  0 Load(1120)
Distance of the route: 14140m
Load of the route: 1120

Total distance of all routes: 42290m
Total load of all routes: 2688


In [15]:
N_items = [58,26,9,11]
weight_items = [20.5,20.4,87.48,16.5]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]
print(total_demand)
demand = [0]
for i in range(22):
    demand.append(total_demand/22)
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Saturday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

2688.2200000000003
Route for vehicle 0:
 0 Load(0) ->  6 Load(122) ->  5 Load(244) ->  4 Load(366) ->  3 Load(488) ->  2 Load(610) ->  1 Load(732) ->  0 Load(732)
Distance of the route: 0m
Load of the route: 732

Route for vehicle 1:
 0 Load(0) ->  22 Load(122) ->  21 Load(244) ->  20 Load(366) ->  19 Load(488) ->  18 Load(610) ->  17 Load(732) ->  16 Load(854) ->  15 Load(976) ->  14 Load(1098) ->  13 Load(1220) ->  12 Load(1342) ->  11 Load(1464) ->  10 Load(1586) ->  9 Load(1708) ->  8 Load(1830) ->  7 Load(1952) ->  0 Load(1952)
Distance of the route: 0m
Load of the route: 1952

Total distance of all routes: 0m
Total load of all routes: 2684


In [9]:
N_items = [72,29,15,22]
weight_items = [20.5,20.4,87.48,16.5]
min_place = [90,90,90,90,20,30]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]

demand = [0]
for i in range(len(min_place)):
    demand.append(total_demand*min_place[i]/sum(min_place))
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Tuesday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

Route for vehicle 0:
 0 Load(0) ->  4 Load(821) ->  6 Load(1094) ->  1 Load(1915) ->  0 Load(1915)
Distance of the route: 14500m
Load of the route: 1915

Route for vehicle 1:
 0 Load(0) ->  2 Load(821) ->  5 Load(1003) ->  3 Load(1824) ->  0 Load(1824)
Distance of the route: 8100m
Load of the route: 1824

Total distance of all routes: 22600m
Total load of all routes: 3739


In [11]:
N_items = [72,29,15,22]
weight_items = [20.5,20.4,87.48,16.5]
min_place = [90,90,90,90,20,20,30]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]

demand = [0]
for i in range(len(min_place)):
    demand.append(total_demand*min_place[i]/sum(min_place))
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Thursday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

Route for vehicle 0:
 0 Load(0) ->  4 Load(783) ->  3 Load(1566) ->  7 Load(1827) ->  0 Load(1827)
Distance of the route: 16500m
Load of the route: 1827

Route for vehicle 1:
 0 Load(0) ->  6 Load(174) ->  5 Load(348) ->  2 Load(1131) ->  1 Load(1914) ->  0 Load(1914)
Distance of the route: 8350m
Load of the route: 1914

Total distance of all routes: 24850m
Total load of all routes: 3741


In [13]:
N_items = [72,29,15,22]
weight_items = [20.5,20.4,87.48,16.5]
min_place = [90,90,90,90,20]
total_demand = 0
for i in range(len(N_items)):
    total_demand += N_items[i]*weight_items[i]

demand = [0]
for i in range(len(min_place)):
    demand.append(total_demand*min_place[i]/sum(min_place))
demand_int = [int(i) for i in demand]

df = pd.read_excel('Matrix.xlsx', sheet_name='Friday', index_col=0)

list_2d = df.values.tolist()    
capacities = [2000, 2000]

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = list_2d
    data['demands'] = demand_int
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

main()

Route for vehicle 0:
 0 Load(0) ->  2 Load(886) ->  4 Load(1772) ->  0 Load(1772)
Distance of the route: 12500m
Load of the route: 1772

Route for vehicle 1:
 0 Load(0) ->  1 Load(886) ->  3 Load(1772) ->  5 Load(1968) ->  0 Load(1968)
Distance of the route: 11800m
Load of the route: 1968

Total distance of all routes: 24300m
Total load of all routes: 3740
